### scraping methods
1. Session initialization <br>
2. Extracting links that lead to content, given some search query <br>
3. Extracting content <br>
4. Main function, that is used to extract all navigation links and all content. <br>

In [6]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup as bs
import json
import lxml.html

links = []
pictures = []
videos = []

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

#open the link to @Bee_nfluencer profile which I plan to discover 
username='bee_nfluencer'
mainUrl = 'https://www.instagram.com/'+username+'/?hl=en'
driver.get(mainUrl)
time.sleep(5)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
There is no [win32] chromedriver for browser 93.0.4577 in cache
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\alloy\.wdm\drivers\chromedriver\win32\93.0.4577.63]


In [11]:
from instascrape import Post, Profile
import os
import sys
import time
from sys import platform

import numpy as np
from bs4 import BeautifulSoup
import shutil



def profile_push_button(browser):
    try:
        driver.find_element_by_css_selector("#react-root > section > main > div > div._2z6nI > div.Igw0E.rBNOH.eGOV_._4EzTm > div > button").click()
    except Exception:
        print('no button found')
    else:
        print('button clicked')
    time.sleep(5)
    
def profile_scroll_page(profile, browser, delay):
    browser.get(profile.url)

    source_data = []
    js_script = (
            "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;"
        )
    profile_push_button(browser)
    lenOfPage = browser.execute_script(js_script)
    
    match = False
    while not match:
        #time.sleep(abs(np.random.normal(3, 1.5)))
        lastCount = lenOfPage
        time.sleep(delay)
        lenOfPage = browser.execute_script(js_script)
        time.sleep(delay)
        lenOfPage = browser.execute_script(js_script)
        time.sleep(delay)
        lenOfPage = browser.execute_script(js_script)
        source_data.append(browser.page_source)
        if lastCount == lenOfPage:
            match = True
            print(lastCount)
    return source_data, browser

def profile_seperate_posts(profile, source_data):

    post_soup = []
    for source in source_data[0]:
        soup = BeautifulSoup(source, features="lxml")
            # posts = soup.find("span", {"id":"react-root"})

        anchor_tags = soup.find_all("a")
        found = [tag for tag in anchor_tags if tag.find("div", {"class": "eLAPa"})]

        for tag in found:
            if tag not in post_soup:
                post_soup.append(tag)
    return post_soup

def profile_create_post_objects(profile, post_soup, post_size):
        # create post objects for each post on the page
    profile_posts = []
    for post in post_soup:
        shortcode = post["href"].replace("/p/", "")[:-1]
        profile_posts.append(Post(shortcode))
        if len(profile_posts) >= post_size:
            break
            
    return profile_posts

def profile_grab_useful_data(profile_posts):
    for i, post in enumerate(profile_posts):
        if i % 10 == 0:
            print("Read {} posts".format(i))
        try:
            post.scrape()
        except Exception as e:
            print(e)
                
def dynamic_load(profile, browser, max_posts=sys.maxsize, delay=3):
        source_data = profile_scroll_page(profile, browser, delay)
        post_soup = profile_seperate_posts(profile, source_data)
        profile_posts = profile_create_post_objects(profile, post_soup, max_posts)
        profile_grab_useful_data(profile_posts)
        return profile_posts
        #browser.close()

max_posts_to_load = 500
mydelay = 2

my_prof = Profile(username)
my_prof.scrape()
print(my_prof.url)
myprof_posts = dynamic_load(my_prof, driver, max_posts=max_posts_to_load, delay=mydelay)
print("that's all")
print(myprof_posts)


data_arr = []
counter = 0
for post in myprof_posts:
    try:
        counter = counter + 1
        post.download("/Users/a/Desktop/py_web_scrap/posts_visual_content/" + str(post.upload_date) + ".png")
        #I'll also create binary code of image in the post by following 3 lines
        with open("/Users/a/Desktop/py_web_scrap/posts_visual_content/" + str(post.upload_date) + ".png","rb") as image:
            binaryf = Binary(image.read())
            data_arr.append((post.url, post.upload_date, post.likes, post.comments, post.caption, post.tagged_users, post.hashtags, binaryf))    
    except AttributeError as e: 
        pass 
    
def to_csv_post_details(dataframe):
    dataframe.to_csv('post_details.csv', header = True)


dataframe = pd.DataFrame(data_arr, columns=['url', 'datetime', 'likes', 'comments', '1st comment', 'tagged', 'hashtags', 'binary_img'])
dataframe = dataframe.sort_values(by=['datetime']).reset_index(drop=True)   #Sort by date
dataframe  

to_csv_post_details(dataframe)

ImportError: cannot import name 'extract_email' from 'helpers' (C:\opt\conda\envs\mya\lib\site-packages\helpers\__init__.py)